In [ ]:
import panel as pn
import pandas as pd
import matplotlib.pyplot as plt

from bokeh.sampledata import stocks

pn.extension()

This example is meant to make it easy to compare and contrast the different APIs Panel provides to declare apps and dashboards. Specifically it compares four different implementations of the same app using the 1) ``interact`` function, 2) reactive functions 3) explicit callbacks and 4) a param based implementation.

Before comparing the different approaches we will first declare some components of the app that will be shared, which includes the title of the app, the available stock tickers, a function to return a dataframe given the stock ``ticker`` and the rolling mean ``window_size`` and another function to return a plot given those same inputs:

In [ ]:
title = '## Stock Explorer Matplotlib'

tickers = ['AAPL', 'FB', 'GOOG', 'IBM', 'MSFT']

def get_df(ticker, window_size):
    df = pd.DataFrame(getattr(stocks, ticker))
    df['date'] = pd.to_datetime(df.date)
    return df.set_index('date').rolling(window=window_size).mean().reset_index()

def get_plot(ticker, window_size):
    df = get_df(ticker, window_size)
    df.plot.line('date', 'close', figsize=(10, 6))
    fig = plt.gcf()
    plt.close()
    return fig

### Interact
In the ``interact`` model the widgets are automatically generated from the arguments to the function or by providing additional hints to the ``interact`` call. This is an easy way to generate a simple app but it is not very explicit and to compose the different components in a custom way it is necessary to unpack the layout returned by the ``interact`` call:

In [ ]:
interact = pn.interact(get_plot, ticker=tickers, window_size=(1, 21, 5))

pn.Row(
    pn.Column(title, interact[0]),
    interact[1]
)

### Reactive

The reactive programming model is very similar to the ``interact`` function but makes it possible to explicitly declare the inputs to the function using the ``depends`` decorator and makes the layout of the different components more explicit. In principle we could reuse the ``get_plot`` function from above here and wrap it in the decorator but to be clearer we will repeat it:

In [ ]:
ticker = pn.widgets.Select(name='Ticker', options=tickers)
window = pn.widgets.IntSlider(name='Window Size', value=6, start=1, end=21)

@pn.depends(ticker.param.value, window.param.value)
def get_plot(ticker, window_size):
    df = get_df(ticker, window_size)
    df.plot.line('date', 'close', figsize=(10, 6))
    fig = plt.gcf()
    plt.close()
    return fig

pn.Row(
    pn.Column(title, ticker, window),
    get_plot
)

### Parameterized class

Another similar approach expresses the app entirely as a single ``Parameterized`` class with parameters, which are independent of any GUI code, declaring the inputs. Once again the ``depends`` decorator is used to express the dependencies but in this case the dependencies are expressed as strings referencing the parameters on the class itself. The parameters and the ``plot`` method can then be layed out independently.

In [ ]:
import param

class StockExplorer(param.Parameterized):
    
    ticker = param.Selector(default='AAPL', objects=tickers)
    
    window_size = param.Integer(default=6, bounds=(1, 21))
    
    @param.depends('ticker', 'window_size')
    def plot(self):
        return get_plot(self.ticker, self.window_size)
    
explorer = StockExplorer()
pn.Row(explorer.param, explorer.plot)

### Callbacks

Declaring an app through explicit callbacks can often afford the most control about precisely how the different components of the app are updated, however it quickly grows in complexity the more callbacks there are and therefore can easily become unmaintainable. The approach works by defining callbacks using the ``.param.watch`` API which either update or replace the already rendered components.

In [ ]:
ticker = pn.widgets.Select(name='Ticker', options=['AAPL', 'FB', 'GOOG', 'IBM', 'MSFT'])
window = pn.widgets.IntSlider(name='Window', value=6, start=1, end=21)

row = pn.Row(
    pn.Column(title, ticker, window),
    get_plot(ticker.options[0], window.value)
)

def update(event):
    row[1].object = get_stock(ticker.value, window.value)

ticker.param.watch(update, 'value')
window.param.watch(update, 'value')

row.servable()